In [ ]:
%load_ext autoreload
%autoreload 2
%store -r the_page

In [ ]:
page_name = the_page.page_id #'Chicago'
from external.wikipedia import WikipediaDV, WikipediaAPI
wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))
page = wikipedia_dv.get_page(page_name)

# Editions per page with Wikiwho

In [ ]:
from wikiwho_wrapper import WikiWho
import pandas as pd
import qgrid
# set the default max number of rows to 10 so the larger DataFrame we render don't take up to much space 
qgrid.set_grid_option('maxVisibleRows', 5)


wikiwho = WikiWho(lng='en')
editions = wikiwho.dv.editions(the_page.page_id)

qgrid.show_grid(editions)

In [ ]:
editions['year_month'] = pd.to_datetime(editions['year_month'])

# Group the data by year month and page (drop the editor information)
editions.drop('editor_id', axis=1).groupby(['year_month','page_id']).sum()

# add columns with the total actions
editions = editions.join(pd.DataFrame(
    editions.loc[:,'adds':'adds_stopword_count'].values +\
    editions.loc[:,'dels':'dels_stopword_count'].values +\
    editions.loc[:,'reins':'reins_stopword_count'].values, 
    index=editions.index, 
    columns=['actions', 
             'actions_surv_48h', 
             'actions_persistent', 
             'actions_stopword_count']
))

# Visualization
from visualization.editions_listener import DFListener
from ipywidgets import interact
listener = DFListener(editions)
actions = (editions.loc[:,'actions':'actions_stopword_count'].columns.append(
    editions.loc[:,'adds':'reins_stopword_count'].columns)).values.tolist()


interact(listener.editions_per_month, 
         begin=editions.year_month,
         end=editions.year_month.sort_values(ascending=False),
         granularity=['Yearly', 'Monthly'],
         black=actions, red=['actions_surv_48h'] + actions + ['None'],
         green=['None'] + actions, blue=['None'] + actions)


# Conflict Score of Editors of a Page

In [ ]:
from metrics.conflict import ConflictCalculator
from wikiwho_wrapper import WikiWho

wikiwho = WikiWho(lng='en')
calculator = ConflictCalculator(the_page.page_id, wikiwho)
calculator.calculate()


# Conflict Score of Editors of a Page

In [ ]:
print(f'Total article conflict score: {calculator.get_page_conflict_score()}')
editors_conflicts = calculator.get_conflict_score_per_editor()
editors_conflicts

# Grouping the conflict insertions and deletions per editor

In [ ]:
# to visualize the entire tokens
pd.set_option('display.max_colwidth', -1)

# get the words that each editor introduce and delete
conflicts = calculator.get_all_conflicts()
if len(conflicts) > 1:
    conflict_tokens = conflicts[['token', 'editor', 'action']].groupby(
        ['editor', 'action']).aggregate(lambda x: set(x))

    # merge back with the previous table so one can see the conflict score
    conflict_tokens.reset_index(1).join(
        editors_conflicts['conflict_score']).set_index('action', append=True).sort_values(
        'conflict_score', ascending=False)

else:
    print("There are no conflicts in this article")

# Selecting a controversial editor 

In [ ]:
def editor_fn(editor):
    global the_editor
    global editor_inputname
    editor_inputname = editor
    
    
    wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))
    try:
        the_editor = wikipedia_dv.get_editor(int(editor))
    except:
        the_editor = wikipedia_dv.get_editor(editor[2:])

    # display the data that will be passed to the next notebook
    display(the_editor.to_frame('values'))
    
    %store the_editor
    %store editions
    %store editor_inputname
    %store calculator
       
if len(editors_conflicts) > 0:
    from IPython.display import display, Markdown as md
    display(md('**Select an editor based on the previous information**'))
    display(md('*Recomendation: editors with many conflicts and moderate conflict score*'))
    interact(editor_fn, editor=editors_conflicts.index)
else:
    print("There are no conflicts")

In [ ]:
if len(editors_conflicts) > 0:
    from IPython.display import HTML
    HTML('<a href="3. Using WikiWho to analyze an editor in the context of a page.ipynb" target="_blank">Go to next workbook</a>')
else:
    from IPython.display import HTML
    display(HTML('<a href="1. General Metadata of a Wikipedia Article.ipynb" target="_blank">Go back to the previous workbook</a>'))